In [1]:
import tensorflow as tf
import pandas as pd
import os

2024-05-26 19:07:37.572280: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 19:07:37.572422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 19:07:37.754437: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def parse_function(proto):
    feature_description = {
    'x': tf.io.FixedLenFeature([556], tf.float32),
    'targets': tf.io.FixedLenFeature([368], tf.float32)
    }     
    parsed_features = tf.io.parse_single_example(proto, feature_description)
    features = parsed_features['x']
    targets = parsed_features['targets']
    return features, targets

In [5]:
N_elements_per_tfrecord = 100000
file_path_data = "/kaggle/input/leap-train-tfrecords/"
train_files = [os.path.join(file_path_data, "train_%.3d.tfrec" % i) for i in range(100)]
valid_files = [os.path.join(file_path_data, "train_%.3d.tfrec" % i) for i in range(100, 101)]




In [9]:
BATCH_SIZE=4024
def build_dataset(filepaths):
    return tf.data.TFRecordDataset(filepaths).shuffle(
        buffer_size=2048).map(
        map_func=parse_function,num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(
        batch_size=BATCH_SIZE,drop_remainder=True).cache().repeat().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


train_set = build_dataset(train_files)
valid_set = build_dataset(valid_files)



In [10]:
# Create normalization layer and adapt it to the training data
normalization_layer = tf.keras.layers.Normalization()
sample_data = train_set.map(lambda x, y: x).take(40)  # Take a sample of 20 batches to adapt the normalization layer
normalization_layer.adapt(sample_data)

# Define the model
model = tf.keras.Sequential([
    normalization_layer,
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(256,activation="relu"),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(368)  # Adjust the output shape as needed
])


In [11]:
# Learning rate scheduler with warmup and cosine decay
import numpy as np

# Learning rate scheduler with exponential decay
initial_learning_rate = 1e-3
decay_steps = 100000  # Number of steps before the learning rate is decayed
decay_rate = 0.96  # The rate at which the learning rate decays

lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True  # If True, decay the learning rate at discrete intervals
)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_scheduler),
    loss='mean_squared_error',
    metrics=['mean_absolute_error']
)


In [12]:
steps_per_epoch = int(np.ceil(len(train_files) * 100000 / BATCH_SIZE))

# Fit the model
history = model.fit(
    train_set,
    validation_data=valid_set,
    epochs=10,  # Total number of epochs
    steps_per_epoch=steps_per_epoch,
    validation_steps=steps_per_epoch // 10  # Adjust validation steps as needed
)

Epoch 1/10
 460/2486 ━━━━━━━━━━━━━━━━━━━━ 11:31 341ms/step - loss: 183.0621 - mean_absolute_error: 2.1030

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'], color='tab:blue')
plt.plot(history.history['val_loss'], color='tab:red')
plt.yscale('log')